In [14]:
from requests_oauthlib import OAuth1
import json
import sys
import os
import requests
import mysql.connector
import time
from lxml import etree
import re
#from lxml import etree
#import re

In [13]:
import nltk
from nltk.corpus import stopwords

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [17]:
mydb = mysql.connector.connect(
  host="localhost",user="root",passwd="H9Xho5pMtOTvcQ9N",database="bilibili"
)

In [37]:
CACHE_FNAME = 'proj_cache.json'
try:
    cache_file = open(CACHE_FNAME, 'r')
    cache_contents = cache_file.read()
    CACHE_DICTION = json.loads(cache_contents)
    cache_file.close()

# if there was no file, no worries. There will be soon!
except:
    CACHE_DICTION = {}
def make_request_using_cache(url,headers=None,params=None):
    unique_ident = url
    ## first, look in the cache to see if we already have this data
    if unique_ident in CACHE_DICTION:
        print("Getting cached data...")
        return CACHE_DICTION[unique_ident]
    else:
        print("Making a request for new data...")
        # Make the request and cache the new data
        resp = requests.get(url,headers=headers,params=params)
        CACHE_DICTION[unique_ident] = resp.text
        dumped_json_cache = json.dumps(CACHE_DICTION)
        fw = open(CACHE_FNAME, "w")
        fw.write(dumped_json_cache)
        fw.close()  # Close the open file
        return CACHE_DICTION[unique_ident]

In [18]:
mycursor = mydb.cursor()

In [ ]:
def get_page(page):
    try:
        #这里要加上cookie
        headers = {
            'Cookie': 'LIVE_BUVID=AUTO5015218915110407; sid=4oag5i0u; fts=1521891539; pgv_pvi=3655305216; UM_distinctid=16257cdfffd2e4-032750a28294b2-3b60450b-100200-16257cdfffe2a0; buvid3=7B94813D-1039-4A88-A1EE-9AEFDF54BE05140244infoc; rpdid=kxsliqpkisdosikxllmww; CURRENT_QUALITY=80; finger=edc6ecda',
            'Host': 'search.bilibili.com',
            'Referer': 'https://www.bilibili.com/',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
        }
        url = 'https://search.bilibili.com/all?keyword=python&from_source=banner_search&spm_id_from=333.334.banner_link.1&page={}'.format(str(page))
        response = make_request_using_cache(url,headers=headers)
        if response.status_code == 200:
            return response.text
    except Exception:
        return None

def xpath_parse_page(html):
    data = etree.HTML(html)
    items = data.xpath('//*[@class="video-contain clearfix"]/li')
    for item in items:
        yield {
        'video_image' : item.xpath('./a/div/div[1]/img/@src')[0],
        'video_title' : item.xpath('./div/div[1]/a/text()')[0],
        'video_view' : item.xpath('./div/div[3]/span[1]/text()')[0].strip(),
        'video_updateTime' : item.xpath('./div/div[3]/span[3]/text()')[0].strip(),
        'video_up' : item.xpath('./div/div[3]/span[4]/a/text()')[0]
        }

In [19]:
def fetch_rank():
    baseurl = "https://api.bilibili.com/x/web-interface/ranking/region"
    params = {"jsonp": "jsonp", "rid": 138,"day":7,"original":0}
    response = make_request_using_cache(baseurl,params=params)
    return response.json()

In [23]:
def fetch_dynamic():
    baseurl = "https://api.bilibili.com/x/web-interface/dynamic/region"
    params = {"jsonp": "jsonp", "rid": 138,"pn":1,"ps":30}
    response = make_request_using_cache(baseurl, params=params)
    return response

In [ ]:
def fetch_video_info(aid):
    baseurl = "https://api.bilibili.com/x/web-interface/archive/stat?aid="+aid
    params = {"jsonp": "jsonp", "rid": 138,"pn":1,"ps":30}
    response = make_request_using_cache(baseurl, params=params)
    return response.json()

In [10]:
def fetch_user(mid):
    baseurl = "http://api.bilibili.com/x/space/acc/info"
    params = {"mid": mid}
    response = make_request_using_cache(baseurl, params=params)
    return response.json()

In [11]:
def fetch_user_fellow(vmid):
    baseurl = "https://api.bilibili.com/x/relation/stat?"
    params = {"vmid": vmid,"jsonp": "jsonp"}
    response = make_request_using_cache(baseurl, params=params)
    return response.json()

In [35]:
def fetch_info_once(mycursor):
    tempResult = fetch_dynamic()
    #print(json.dumps(tempResult))
    for i in json.loads(tempResult)['data']['archives']:
        data_aid = i['aid']
        data_tid = i['tid']
        data_tname = i['tname']
        data_pic = i['pic']
        data_title = i['title']
        data_pubdate = i['pubdate']
        data_desc = i['desc']
        data_mid = i['owner']['mid']
        data_view = i['stat']['view']
        data_danmaku = i['stat']['danmaku']
        data_reply = i['stat']['reply']
        data_favorite = i['stat']['favorite']
        data_coin = i['stat']['coin']
        data_share = i['stat']['share']
        data_like = i['stat']['like']
        data_bvid = i['bvid']
        sql = "SELECT * FROM data1 WHERE aid = %s"
        val = (data_aid,)
        mycursor.execute(sql,val)
        myresult = mycursor.fetchall()
        if len(myresult)==0:
            sql = "INSERT INTO data1 (aid,title,pic,pubdate,miaoshu,mid,view,danmaku,\
reply,favorite,coin,share,dianzan,bvid,tid,tname)\
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            val = (data_aid, data_title,data_pic,data_pubdate,data_desc,data_mid,data_view,data_danmaku,\
           data_reply,data_favorite,data_coin,data_share,data_like,data_bvid,data_tid,data_tname)
            mycursor.execute(sql, val)
            mydb.commit()
            #print(mycursor.rowcount, "record inserted.")
   

In [38]:
count = 5
while count>=0:
    count = count-1
    fetch_info_once(mycursor)
    time.sleep(1)

Making a request for new data...
fetch once
Getting cached data...
fetch once
Getting cached data...
fetch once
Getting cached data...
fetch once
Getting cached data...
fetch once
Getting cached data...
fetch once


In [73]:
count = 100
while count>=0:
    count = count-1
    fetch_info_once(mycursor)
    time.sleep(25)

fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once
fetch once

1 record inserted.


0

In [15]:
for i in tempResult['data']['archives']:
    print(i["title"],",mid:",i['owner']['mid'],",播放：",i['stat']['view'],"点赞",i['stat']['favorite'])

如果男人的标配是有车有房有存款，那女人的标配应该是什么呢？ ,mid: 449590290 ,播放： 608924 点赞 4440
外国妹子入坑动漫后的变化... ,mid: 306477658 ,播放： 1334492 点赞 10195
吊车尾参军那点事（四十八） ,mid: 95480382 ,播放： 30688 点赞 337
【r1se】沙雕男团壶的大型蹦迪现场 ,mid: 12555036 ,播放： 21809 点赞 919
都TM别颠了，醒了！醒了！ ,mid: 5957155 ,播放： 1565052 点赞 9374
【不要笑死挑战！！！】现场整活了兄弟们 ,mid: 14946689 ,播放： 347596 点赞 4317
【我是谁】？？？难度，猜中一个算我输 ,mid: 52973637 ,播放： 335068 点赞 3109
千万不要跟声优斗表情包，否则你将毫无胜算【30】 ,mid: 7064298 ,播放： 432422 点赞 14788
欧洲人学bilibili语？什么高端操作？4 ,mid: 406472610 ,播放： 98377 点赞 963
【东北小朋友模仿英语老师合集】所以老师也是全国统一的吗 ,mid: 99027373 ,播放： 195854 点赞 3407
【恶作剧】洗发水的故事II-这次直接上假血 ,mid: 94898339 ,播放： 781387 点赞 9580
半佛仙人对大专好不好找工作等问答 ,mid: 369066047 ,播放： 1577 点赞 8
【小缸和阿灿】百万up大比拼，为了赢，把小学糗事都说出来了 ,mid: 37961599 ,播放： 613046 点赞 9491
【张氏心脑方】只需一副药，彻底死翘翘，吃完当天就见孝 ,mid: 394201823 ,播放： 781819 点赞 32659
人工智能将会毁灭世界？~韩国神经配音【中字幕】韩国搞笑动画 ,mid: 221607275 ,播放： 15763 点赞 138
自制战靴实战测评！ ,mid: 52982448 ,播放： 96587 点赞 829
国内首个内置虞书欣语音包的人工智能 ,mid: 7189540 ,播放： 595632 点赞 23521
【朱一旦】182 原来这就是传说中的猛男粉 ,mid: 437316738 ,播放： 108172